In [11]:
# Load in the python script containing the same code as the load the data notebook
%run loadData.py
# now we can access train, dev, and test
# along with trainSents, devSents testSents

In [12]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

import nltk
from nltk.corpus import stopwords

import pickle # Useful for read / write of list file
import os #Needed to check if file exists

from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

stop_words = set(stopwords.words('english'))

from string import punctuation  
punct_tokens = set(punctuation)
extra_tokens = set(["what", "where", "how", "when", "who"])

filter_tokens = extra_tokens.union(punct_tokens).union(stop_words)

In [13]:
# Preprocessing tuning functions

# Follow lemmatize function from guide notebook: WSTA_N1B_preprocessing.ipynb
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.WordPunctTokenizer() #word_tokenize #tokenize.regexp.WordPunctTokenizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def pre_process(line):
    tokenized_sentence = word_tokenizer.tokenize(line.lower())
    lemmatized_sentence = [lemmatize(token) for token in tokenized_sentence]
    filtered_sentence = [token for token in lemmatized_sentence if token not in filter_tokens]
    return filtered_sentence

In [14]:
# Core functions

def vectorize_documents(text_documents):

    vectorizer = TfidfVectorizer(stop_words='english', tokenizer=pre_process)
    vector_documents = vectorizer.fit_transform(text_documents)
    
    return [vector_documents, vectorizer]

def vectorize_query(vectorizer, text_query):
    return vectorizer.transform([text_query])

def process_neighbours(vector_documents):
    
    neighbours = NearestNeighbors(1, algorithm="brute", metric="cosine")
    neighbours.fit(vector_documents)
    
    return neighbours

def closest_document(neighbours, vector_query):

    result = neighbours.kneighbors(vector_query, 1, return_distance=True)

    result_index = result[1][0][0]
    result_distance = result[0][0][0]
    
    return [result_distance, result_index]

In [15]:
classifier = './stanford/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = './stanford/stanford-ner.jar'

sTagger = StanfordNERTagger(classifier,jar)

In [16]:
def getStanfordTagging(datasetName):
    fnameTrain = './preCompTags/stanfordTaggedTrain.txt'
    fnameDev = './preCompTags/stanfordTaggedDev.txt'
    fnameTest = './preCompTags/stanfordTaggedTest.txt'
    
    theFilePath = ''
    theSents = []
    if (datasetName == 'train'):
        theFilePath = fnameTrain
        theSents = trainSents
    elif (datasetName == 'dev'):
        theFilePath = fnameDev
        theSents = devSents
    elif (datasetName == 'test'):
        theFilePath = fnameTest
        theSents = testSents
    else :
        raise ValueError('Incorrect datasetName: ' + datasetName + ', choose from - "train", "dev", "test" ') 
    if (os.path.exists(theFilePath)):
        with open(theFilePath, "rb") as fp:
            stanfordTags = pickle.load(fp)
            return stanfordTags
    
    else :
        #Need to create taggings!
        taggedSentsList = []
        for sents in theSents:
            tokenisedSents = [word_tokenize(sent) for sent in sents]
            classifiedSents = sTagger.tag_sents(tokenisedSents)
            taggedSentsList.append(classifiedSents)
        #And save them
        with open(theFilePath, "wb") as fp: 
            pickle.dump(taggedSentsList, fp)
        return taggedSentsList
    

In [17]:
taggedTrain = getStanfordTagging('train')
taggedDev = getStanfordTagging('dev')

In [18]:
# Given a stanford tagged list, refines the list by:\n",
# Grouping all contiguous words with the same tag\n",
# Relabels Organisations as Other\n",
# Labels Number\n",
def refineWordTags(taggedWordList):
    newWordTags = []
    prevWord = ''
    prevTag = taggedWordList[0][1] # Ie the first tag\n",
    for (word, tag) in taggedWordList:
        if (tag == 'ORGANIZATION'):
            tag = 'O'
        if (tag == 'O'):
            #Might be a number
            if isNumber(word):
                tag = 'NUMBER'
            elif isCapitalised(word):
                tag = 'OTHERCAP'
            elif isStopWord(word):
                tag = 'STOPWORD'
        if (tag == prevTag):
            prevWord += ' ' + word
        else :
            newWordTags.append((prevWord, prevTag))
            prevWord = word
            prevTag = tag
    # Need to add the final ones
    newWordTags.append((prevWord, prevTag))
    return newWordTags
        
# Thanks : http://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers\n",
numInWords = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"
       , "hundred", "thousand", "million", "billion", "trillion"]

def isCapitalised (word):
    if len(word) == 0:
        return False
    return word[0].isupper()

# Returns true if the word represents a number\n",
def isNumber(word):
    pattern = ".?(\\d)+((,|.)(\\d)+)*"
    if re.match(pattern,word) :
        return True
    if word.lower() in numInWords:
        return True
    return False

def isStopWord(word):
    return word.lower() in stop_words


In [19]:
# For each question, evaluate if the answer is present as an entity

def evaluateNER(questionsList, documentsList, numToEval):
    
    correct = []
    wrong = []
    
    for i in range (0, numToEval):
        
        documents = documentsList[i]
        questions = questionsList[i]
        
        for j in range (0, len(questions)):
            
            answer = questionsList[i][j]["answer"]
            answerID = questionsList[i][j]["answer_sentence"]
            possAnswers = refineWordTags(taggedDev[i][answerID])
            inThere = False
            
            for possAnswer in possAnswers:
                if possAnswer[0] == answer:
                    inThere = True
                    break
            if inThere:
                correct.append((i,j, possAnswers))
            else:
                wrong.append((i,j, answer, possAnswers))

    return (correct, wrong)


In [20]:
(corNER, wrongNER) = evaluateNER(dev, devSents,len(dev))

print("Number Correct : " + str(len(corNER)))
print("Number incorrect: " + str(len(wrongNER)))
print("Average correct : " + str((len(corNER) + 0.0) / (len(corNER)+len(wrongNER))))

Number Correct : 3408
Number incorrect: 5055
Average correct : 0.402694080113


In [45]:
# Part A gives us a most likely sentence
# Part B splits into entities

# Given a question, returns a tag for the answer form
# From PERSON, LOCATION, NUMBER, OTHER 
# Assuming question is lowercased
def getQuestionType(question):
    if 'Who' in question:
        return "PERSON"
    if 'where' in question:
        return "LOCATION"
    if 'How many' in question:
        return "NUMBER"
    if 'How much' in question:
        return "NUMBER"
    if 'When' in question:
        return "NUMBER"
    if 'what year' in question:
        return "NUMBER"
    else:
        return "O"

In [46]:
def filter_entitites(entities):
    return filter(lambda x: x[0] == "O" or x[0] == "STOPWORD", entities)

In [47]:
# First, answers whose content words all appear in the question should be ranked lowest.

def first_filter2(question, answer_entities):
   
    ranked_list = []
    
    question = set(pre_process(question))
    
#     print question
#     print
    
    for entity in answer_entities:

        raw_span = entity[0]
        span_tag = entity[1]
        
        set_span = set(pre_process(raw_span))
        
        if span_tag != "O" and span_tag != "STOPWORD" and span_tag !="O":
            
            if set_span.issubset(question):
                
                ranked_list.append([entity, 1])
#                 print "IN", raw_span, span_tag, set_span, question
                
            else:
                
                ranked_list.append([entity, 2])
#                 print "OUT", raw_span, span_tag, set_span, question
    
    return sorted(ranked_list, key=lambda x: x[1], reverse=True)

In [48]:
# Second, answers which match the question type should be ranked higher than those that don't; for this, you
# should build a simple rule-based question type classifier based on key words (e.g. questions which contain "who" are
# people).

# First, answers whose content words all appear in the question should be ranked lowest.

def second_filter2(question, ranked_list):
   
    question_type = getQuestionType(question)
#     print question_type
    
    for index, answer in enumerate(ranked_list):
        
        entity_tag = answer[0][1]
        
        if entity_tag == question_type:
#             print "MATCH", answer[0], question_type, question
            ranked_list[index].append(2)
#             ranked_list[index][1] += 1
        else:
            ranked_list[index].append(1)
#             ranked_list[index][1] -= 1
            
    return ranked_list

In [49]:
# import nltk



In [50]:
# Follow lemmatize function from guide notebook: WSTA_N1B_preprocessing.ipynb
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
word_tokenizer = nltk.tokenize.WordPunctTokenizer() #word_tokenize #tokenize.regexp.WordPunctTokenizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

def pre_process2(line):
    tokenized_sentence = word_tokenizer.tokenize(line.lower())
    lemmatized_sentence = [lemmatize(token) for token in tokenized_sentence]
    filtered_sentence = [token for token in lemmatized_sentence if token not in filter_tokens]
    tagged_sent = nltk.pos_tag(lemmatized_sentence)
    final = []
    for word, tag in tagged_sent:
        if "V" in tag or "NN" in tag:
#             final.append((word,tag))
            final.append(word)
            
#     print "RESULT: ", final
    return final

In [54]:
# Third, among entities of the same type, the prefered entity should be the one which is closer in the sentence to a
# closed-class word from the question.

from collections import defaultdict

def third_filter2(question, possAnswers, ranked_list):
    
    question = pre_process2(question)

    answer_sent = " ".join([x[0] for x in possAnswers])
    answer_sent = pre_process(answer_sent)
    raw_answer_sent = " ".join(answer_sent)
    
#     print "QUESTION: "
#     pp.pprint(question)
#     print "ANSWER: "
#     pp.pprint(answer_sent)
#     pp.pprint(raw_answer_sent)
    
    for index, answer in enumerate(ranked_list):

        span_tag = answer[0][1]
        raw_span = answer[0][0]

        proc_span = pre_process(raw_span)

        raw_proc_span = " ".join(proc_span)
        new_raw_proc_span = "-".join(proc_span)

        raw_answer_sent = raw_answer_sent.replace(raw_proc_span, new_raw_proc_span)
    
    answer_sent = raw_answer_sent.split(" ")
    
    avg_dict = defaultdict(float)
    
    for open_class in question:
        
        if open_class in answer_sent:
            
            open_class_locations = [i for i, x in enumerate(answer_sent) if x == open_class]
            
#             print "OPEN CLASS: ", repr(open_class)

            for index, answer in enumerate(ranked_list):

                span_tag = answer[0][1]
                raw_span = answer[0][0]

                proc_span = pre_process(raw_span)
                
                raw_proc_span = " ".join(proc_span)
                new_raw_proc_span = "-".join(proc_span)
                
                proc_span_locations = [i for i, x in enumerate(answer_sent) if x == new_raw_proc_span]
                
                min_dist = len(answer_sent)
                min_dist_ind = (None, None)
                
                for loc1 in proc_span_locations:
                    
                    for loc2 in open_class_locations:
                        
                        dist = abs(loc1 - loc2)
                        
                        if dist < min_dist:
                            
                            min_dist = dist
                            min_dist_ind = (loc1, loc2)
                
#                 print "PROC: ", proc_span_locations
#                 print "OPEN CLASS: ", open_class_locations                
                scale = (len(answer_sent) - min_dist) * 1.0 / len(answer_sent)
#                 print "JOINT: ", min_dist_ind, scale
                avg_dict[index] += scale
#                 ranked_list[index][1] *= scale
    
    
    for key, value in avg_dict.iteritems():
        ranked_list[key].append(value / len(question))

    return ranked_list

In [60]:
def reduce_rank(ranking_list):
    
    new_ranking = []
    
    for rank in ranking_list:
        
        new_rank = ( rank[1] + rank[2] )
        
        if len(rank) == 4:
             new_rank *= rank[3]
        
        new_ranking.append([rank[0], new_rank])
        
    return sorted(new_ranking, key=lambda x: x[1], reverse=True)

In [61]:
# For each question, evaluate if the answer is present as an entity

def evaluateAnswerRanking2(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    
    (corNER, wrongNER) = evaluateNER(questionsList,documentsList, numToEval)
    
    entityListsWithAnswer = corNER
    
    for index, (i, j, possAnswers) in enumerate(entityListsWithAnswer):
        
        question = questionsList[i][j]["question"]
        answer =  questionsList[i][j]["answer"]
        
        first_pass = first_filter2(question, possAnswers)
        second_pass = second_filter2(question, first_pass)
        third_pass = third_filter2(question, possAnswers, second_pass)
        fourth_pass = reduce_rank(third_pass)
        
        pp.pprint(fourth_pass)
        print
    
        if index > 1:
            break

In [62]:
evaluateAnswerRanking2(dev, devSents, len(dev))

[   [(u'Leaves', 'OTHERCAP'), 1.3548387096774193],
    [(u'IR-filter', 'OTHERCAP'), 1.2096774193548385],
    [(u'IR', u'LOCATION'), 0.967741935483871],
    [(u'IR-passing', 'OTHERCAP'), 0.629032258064516],
    [(u'Wood', 'OTHERCAP'), 0.3870967741935484],
    [(u'IR-glowing', 'OTHERCAP'), 0.24193548387096775]]

[[(u'The', 'OTHERCAP'), 0.0]]

[   [(u'William Herschel', u'PERSON'), 2.888888888888889],
    [(u'19th', 'NUMBER'), 1.1666666666666665],
    [(u'The', 'OTHERCAP'), 0.0]]



In [63]:
# For each question, evaluate if the answer is present as an entity

def evaluateAnswerRanking(questionsList,documentsList, numToEval):
    
    correct = []
    wrong = []
    
    (corNER, wrongNER) = evaluateNER(questionsList,documentsList, numToEval)
    entityListsWithAnswer = corNER
    
    for (i,j,possAnswers) in entityListsWithAnswer:
        
        question = questionsList[i][j]["question"]
        answer =  questionsList[i][j]["answer"]
        
        first_pass = first_filter2(question, possAnswers)
        second_pass = second_filter2(question, first_pass)
        third_pass = third_filter2(question, possAnswers, second_pass)
        fourth_pass = reduce_rank(third_pass)
                
#         pp.pprint(third_pass)

        top_answer = fourth_pass.pop(0)
#         pp.pprint(top_answer)        
        answerPredicited = top_answer[0][0]
        
#         print answerPredicited

        if (answerPredicited == answer):
            correct.append((i,j))
        else :
            wrong.append((i,j,answerPredicited))
#         break
        #print correct
    return (correct, wrong)
    

In [64]:
(corAns, wrongAns) = evaluateAnswerRanking(dev, devSents,len(dev))
print("Number Correct : " + str(len(corAns)))
print("Number incorrect: " + str(len(wrongAns)))
print ("Average correct : " + str((len(corAns) + 0.0) / (len(corAns)+len(wrongAns))))

Number Correct : 1707
Number incorrect: 1701
Average correct : 0.50088028169


In [481]:
#first:  whose anwsers all appear in the questions rank the lowest
#assumption: input question in its dictionary value
from string import punctuation  
non_words = list(punctuation)

def first_filter(question, anwser_entities):
    ranking_dict_1 = {}
    ranking_list = []
    merge_list = []
    answer_entities_list = []
    for entity in anwser_entities:
        answer_entities_list.append(entity[0])
        if entity[0] in question:
            if entity[0].lower() not in stop_words and entity[0]!='' and entity[0].lower() not in non_words:
                #print entity[0]
                merge_list.append(entity[0])
        else:
            if entity[0].lower() not in stop_words and entity[0]!='' and entity[0].lower() not in non_words:
                ranking_list.append(entity)
    ranking_dict_1["ranking_list"] = ranking_list
    ranking_dict_1["same_word_list"] = merge_list
    ranking_dict_1["answer_entities_list"] = answer_entities_list
    return ranking_dict_1

In [ ]:
#second: answers which match the question type should be ranked higher than those that dont

#assumption: save questions' type in the dictionary format quesiton1 = 
#{u'answer': u'long playing',u'answer_sentence': 2, u'question':......., 'question_type:'PERSON'}

def second_filter(question, ranking_dict_1):
    question_with_type ={}
    question_with_type['question_type']= getQuestionType(question)
    question_with_type['question'] = question
    #print question_with_type
    ranking_dict_2 = {}
    ranking_list =[]
    merge_list = []
    for entity in ranking_dict_1["ranking_list"]:
        if question_with_type['question_type'] == 'O':
            ranking_list.append(entity[0])
        else:
            if entity[1] == question_with_type['question_type']:
                ranking_list.append(entity[0])
            else:
                merge_list.append(entity[0])
    ranking_dict_2["same_word_list"] = ranking_dict_1["same_word_list"]
    ranking_dict_2["ranking_list"] = ranking_list
    ranking_dict_2["Other_tags_list"] = merge_list
    ranking_dict_2["answer_entities_list"] = ranking_dict_1["answer_entities_list"]
    return ranking_dict_2
    

In [ ]:
#Thrid: based on second, the prefered entity should be the one which is close in 
#the sentence to a closed-class word form the question
from collections import OrderedDict

def cal_distance_words(entity,same_words, anwser_entities):
    temp = 0
    for same_word in same_words:
        temp += abs(anwser_entities.index(entity) - anwser_entities.index(same_word))
    return float(temp)/float(len(same_words))

def sort_orderedDict(orderdict):
    return OrderedDict(sorted(orderdict.items(), key = lambda x:x[1], reverse = False))
        

def third_filter(question,second_filter,anwser_entities):
    dict_ranking ={}
    if (len(second_filter["same_word_list"])==0):
        if len(second_filter['ranking_list']) != 0:
            return second_filter['ranking_list'][0]
        else:
            return second_filter["Other_tags_list"][0]
            #for entity in second_filter["Other_tags_list"]:
                #dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
            #dict_ranking = sort_orderedDict(dict_ranking)
            #if len(dict_ranking.items()) ==0:
                #return 0
            #else:
            #return dict_ranking.items()[0][0]
    else:
        #if len(second_filter['ranking_list']) != 0:
        for entity in second_filter["ranking_list"]:
            dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
        #else:
            #if len(second_filter["Other_tags_list"]) !=0:
                #for entity in second_filter["Other_tags_list"]:
                    #dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
            #print dict_ranking
            #else:
                #return 0
        dict_ranking = sort_orderedDict(dict_ranking)
            #print dict_ranking
        if len(dict_ranking.items()) ==0:
            return 0
        else:
            return dict_ranking.items()[0][0]#second: answers which match the question type should be ranked higher than those that dont

#assumption: save questions' type in the dictionary format quesiton1 = 
#{u'answer': u'long playing',u'answer_sentence': 2, u'question':......., 'question_type:'PERSON'}

def second_filter(question, ranking_dict_1):
    question_with_type ={}
    question_with_type['question_type']= getQuestionType(question)
    question_with_type['question'] = question
    #print question_with_type
    ranking_dict_2 = {}
    ranking_list =[]
    merge_list = []
    for entity in ranking_dict_1["ranking_list"]:
        if question_with_type['question_type'] == 'O':
            ranking_list.append(entity[0])
        else:
            if entity[1] == question_with_type['question_type']:
                ranking_list.append(entity[0])
            else:
                merge_list.append(entity[0])
    ranking_dict_2["same_word_list"] = ranking_dict_1["same_word_list"]
    ranking_dict_2["ranking_list"] = ranking_list
    ranking_dict_2["Other_tags_list"] = merge_list
    ranking_dict_2["answer_entities_list"] = ranking_dict_1["answer_entities_list"]
    return ranking_dict_2
    